In [1]:
import pandas as pd
import numpy as np

### Pivoting

In [39]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())

In [40]:
df

,date,variable,value
0,2000-01-03,A,0.443009
1,2000-01-04,A,-0.022290
2,2000-01-05,A,-1.025414
3,2000-01-03,B,-0.790713
4,2000-01-04,B,0.918097
5,2000-01-05,B,0.232675
6,2000-01-03,C,-0.695203
7,2000-01-04,C,-0.980494
8,2000-01-05,C,0.651956
9,2000-01-03,D,-0.975375


In [41]:
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,0.443009,-0.790713,-0.695203,-0.975375
2000-01-04,-0.022290,0.918097,-0.980494,1.116996
2000-01-05,-1.025414,0.232675,0.651956,-0.770526


In [42]:
df['value2'] = df.value*2

In [43]:
df

,date,variable,value,value2
0,2000-01-03,A,0.443009,0.886018
1,2000-01-04,A,-0.022290,-0.044580
2,2000-01-05,A,-1.025414,-2.050827
3,2000-01-03,B,-0.790713,-1.581426
4,2000-01-04,B,0.918097,1.836195
5,2000-01-05,B,0.232675,0.465351
6,2000-01-03,C,-0.695203,-1.390405
7,2000-01-04,C,-0.980494,-1.960989
8,2000-01-05,C,0.651956,1.303912
9,2000-01-03,D,-0.975375,-1.950750


In [45]:
df.pivot(index='date', columns='variable', values='value2')

variable,A,B,C,D
date,,,,
2000-01-03,0.886018,-1.581426,-1.390405,-1.950750
2000-01-04,-0.044580,1.836195,-1.960989,2.233991
2000-01-05,-2.050827,0.465351,1.303912,-1.541052


In [46]:
df.pivot(index='date', columns='variable', values=['value', 'value2'])

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.443009 -0.790713 -0.695203 -0.975375  0.886018 -1.581426   
2000-01-04 -0.022290  0.918097 -0.980494  1.116996 -0.044580  1.836195   
2000-01-05 -1.025414  0.232675  0.651956 -0.770526 -2.050827  0.465351   

                                
variable           C         D  
date                            
2000-01-03 -1.390405 -1.950750  
2000-01-04 -1.960989  2.233991  
2000-01-05  1.303912 -1.541052

### Stacking

In [47]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]))

In [48]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [49]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])

In [50]:
df

A         B
first second                    
bar   one     1.019146 -0.201047
      two    -0.953719 -0.560014
baz   one     1.390845 -0.948742
      two     2.025976  1.029092
foo   one     0.301626  0.498511
      two     1.621891  1.906449
qux   one     0.784476  1.021780
      two     0.045248 -1.394430

In [51]:
df2 = df[:4]

In [52]:
df2

A         B
first second                    
bar   one     1.019146 -0.201047
      two    -0.953719 -0.560014
baz   one     1.390845 -0.948742
      two     2.025976  1.029092

In [53]:
df2.stack() 

first  second   
bar    one     A    1.019146
               B   -0.201047
       two     A   -0.953719
               B   -0.560014
baz    one     A    1.390845
               B   -0.948742
       two     A    2.025976
               B    1.029092
dtype: float64

In [54]:
df2.stack().unstack()

A         B
first second                    
bar   one     1.019146 -0.201047
      two    -0.953719 -0.560014
baz   one     1.390845 -0.948742
      two     2.025976  1.029092

In [55]:
df2.stack().unstack(level=0)

first          bar       baz
second                      
one    A  1.019146  1.390845
       B -0.201047 -0.948742
two    A -0.953719  2.025976
       B -0.560014  1.029092

In [56]:
df2.stack().unstack(level=1)

second        one       two
first                      
bar   A  1.019146 -0.953719
      B -0.201047 -0.560014
baz   A  1.390845  2.025976
      B -0.948742  1.029092

### Melting

In [57]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'], 'last' : ['Doe', 'Bo'], 'height' : [5.5, 6.0], 'weight' : [130, 150]})

In [58]:
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [59]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [62]:
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [63]:
cheese.melt(id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [65]:
cheese.melt(id_vars=['first', 'last'], value_vars=['value'])

C:\Users\lyaa\AppData\Local\Continuum\miniconda3\envs\kaggle\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,first,last,variable,value
0,John,Doe,value,NaN
1,Mary,Bo,value,NaN


In [66]:
cheese.melt(id_vars=['first', 'height'])

,first,height,variable,value
0,John,5.5,last,Doe
1,Mary,6.0,last,Bo
2,John,5.5,weight,130
3,Mary,6.0,weight,150


In [67]:
cheese.melt(id_vars=['first', 'height'], value_vars=['last'])

,first,height,variable,value
0,John,5.5,last,Doe
1,Mary,6.0,last,Bo


In [68]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'}, 'B': {0: 1, 1: 3, 2: 5}, 'C': {0: 2, 1: 4, 2: 6}})

In [69]:
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [70]:
df.melt(id_vars=['A'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [71]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'}, 'B': {0: 1, 1: 3, 2: 5}, 
                   'C': {0: 2, 1: 4, 2: 6}, 'D': {0: 200, 1: 400, 2: 600}})

In [72]:
df

,A,B,C,D
0,a,1,2,200
1,b,3,4,400
2,c,5,6,600


In [74]:
df.melt(id_vars=['A'], value_vars=['B', 'D'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,D,200
4,b,D,400
5,c,D,600


In [79]:
df.melt(id_vars=['A'], value_vars=['B', 'D'], var_name='haha')

,A,haha,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,D,200
4,b,D,400
5,c,D,600


In [78]:
df.melt(id_vars=['A'], value_vars=['B', 'D'], var_name='haha', value_name='baba')

,A,haha,baba
0,a,B,1
1,b,B,3
2,c,B,5
3,a,D,200
4,b,D,400
5,c,D,600


In [80]:
dft = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
   ....:                     "A1980" : {0 : "d", 1 : "e", 2 : "f"},
   ....:                     "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
   ....:                     "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
   ....:                     "X"     : dict(zip(range(3), np.random.randn(3)))
   ....:                    })

In [81]:
dft

,A1970,A1980,B1970,B1980,X
0,a,d,2.5,3.2,-0.409373
1,b,e,1.2,1.3,0.129323
2,c,f,0.7,0.1,-0.313973


In [83]:
dft.melt(id_vars=['A1970', 'A1980', 'B1970', 'B1980'])

,A1970,A1980,B1970,B1980,variable,value
0,a,d,2.5,3.2,X,-0.409373
1,b,e,1.2,1.3,X,0.129323
2,c,f,0.7,0.1,X,-0.313973


In [85]:
dft.melt(id_vars=['X'])

,X,variable,value
0,-0.409373,A1970,a
1,0.129323,A1970,b
2,-0.313973,A1970,c
3,-0.409373,A1980,d
4,0.129323,A1980,e
5,-0.313973,A1980,f
6,-0.409373,B1970,2.5
7,0.129323,B1970,1.2
8,-0.313973,B1970,0.7
9,-0.409373,B1980,3.2
